# SteelScript NetProfiler HostGroup Importing

## Imports and Setup

In [ ]:
import csv
from collections import defaultdict

In [ ]:
import steelscript
from steelscript.netprofiler.core.netprofiler import NetProfiler
from steelscript.common.service import UserAuth

from steelscript.netprofiler.core.hostgroup import HostGroupType, HostGroup
from steelscript.commands.steel import prompt_yn
from steelscript.common.exceptions import RvbdException

In [ ]:
hostname = "NETPROFILER.HOSTNAME.COM"
username = "USERNAME"
password = "PASSWORD"

In [ ]:
host = hostname
auth = UserAuth(username, password)

## Initialize NetProfiler Object

In [ ]:
p = NetProfiler(host, auth=auth)

## Import Hostgroup File

### Create an example file using the required format and save it to temp directory.

In [ ]:
EXAMPLE = """\
"subnet","SiteName"
"10.143.58.64/26","CZ-Prague-HG"
"10.194.32.0/23","MX-SantaFe-HG"
"10.170.55.0/24","KR-Seoul-HG"
"10.234.9.0/24","ID-Surabaya-HG"
"10.143.58.63/23","CZ-Prague-HG"
"""

example_file = '/tmp/example_groups.txt'

with open(example_file, 'w') as f:
    f.writelines(EXAMPLE)

### Define an import function to read and process the input file.

In [ ]:
def import_file(input_file):
    """Process the input file and load into dict."""
    groups = defaultdict(list)

    with open(input_file, 'rb') as f:
        reader = csv.reader(f)
        header = reader.next()
        if header != ['subnet', 'SiteName']:
            print 'Invalid file format'
            print 'Ensure file has correct header.'
            print 'example file:'
            print EXAMPLE

        for row in reader:
            cidr, group = row
            groups[group].append(cidr)

    return groups

In [ ]:
groups = import_file(example_file)
groups

### Post Hostgroup Updates to NetProfiler

In [ ]:
def update_hostgroups(netprofiler, hostgroup, groups):
    """Replace existing ``hostgroup`` with contents of ``groups`` dict."""
    # First find any existing HostGroupType and delete it.
    try:
        hgtype = HostGroupType.find_by_name(netprofiler, hostgroup)
        print ('Deleting existing HostGroupType "%s".' % hostgroup)
        hgtype.delete()
    except RvbdException:
        print 'No existing HostGroupType found, will create a new one.'
        pass

    # Create a new one
    hgtype = HostGroupType.create(netprofiler, hostgroup)

    # Add new values
    for group, cidrs in groups.iteritems():
        hg = HostGroup(hgtype, group)
        hg.add(cidrs)

    # Save to NetProfiler
    hgtype.save()
    print 'Saved HostGroupType "%s".' % hostgroup

In [ ]:
update_hostgroups(p, 'TestHostGroup', groups)

# Complete Script

<font color='blue'>
Save to file 'import_hostgroups.py' and execute as follows:
</font>

    python import_hostgroups.py <netprofiler> -u <username> -p <password> --hostgroup <hg_name> -i <hostgroup_filepath>